In [26]:
import numpy as np
import pandas as pd
from utils import *
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from copy import copy
import torch.optim as optim
from tqdm import tqdm
df = pd.read_csv('C:\\Users\\LENOVO\\Downloads\\metadata_VAE_v2.csv', index_col=0)
df['Date']=[i[:-2] for i in df['Date'].values]
a = ['29_marzo',
 '14_abril',
 '28_abril',
 '7_mayo',
 '12_mayo',
 '19_mayo',
 '26_mayo',
 '2_junio',
 '11_junio',
 '16_junio',
 '23_junio',
 '2_julio',
 '9_julio',
 '14_julio',
 '23_julio',
 '5_agosto',
 '13_agosto',
 '19_agosto',
 '15_setiembre',
 '24_setiembre',
 '15_octubre',
 '29_octubre',
 '12_noviembre',
 '26_noviembre']
flies_dict = {j:i for i, j in enumerate(a)}
df.Date.replace(flies_dict, inplace=True)
df.iloc[:,-1], df.iloc[:,-2] = df.iloc[:,-2], df.iloc[:,-1]
colist = list(df)
colist[-1], colist[-2] = colist[-2], colist[-1]
df.columns = colist

In [29]:
df

,meta_u_0,meta_u_1,meta_u_2,meta_u_3,meta_u_4,meta_u_5,meta_u_6,meta_u_7,meta_u_8,meta_u_9,...,meta_sig_12,meta_sig_13,meta_sig_14,meta_sig_15,meta_sig_16,meta_sig_17,meta_sig_18,meta_sig_19,Date,Class
0,0.078699,0.000000,0.142968,0.291437,0.194801,0.252386,0.280699,0.194773,0.000000,0.000000,...,0.003038,0.000000,0.586983,0.000000,0.354402,0.142688,0.515514,0.648625,8,N_Control
1,0.608770,0.000000,0.000000,1.317309,0.859336,0.470032,0.406602,0.933932,0.550570,0.000000,...,0.000000,0.000000,0.851890,0.771395,0.319740,0.000000,1.002410,0.462932,8,P_Control
2,0.686334,0.000000,0.000000,1.278347,0.952391,0.331739,0.410137,0.944420,0.512473,0.000000,...,0.000000,0.000000,0.757310,1.089789,0.162783,0.000000,0.765707,0.417435,8,K_Control
3,0.844607,0.000000,0.000000,1.487970,1.042941,0.056912,0.388496,1.284950,0.947870,0.000000,...,0.000000,0.480788,0.872483,1.191065,0.122678,0.000000,1.066383,0.149918,8,Control
4,0.137993,0.000000,0.000000,0.649189,0.256893,0.472469,0.317355,0.611884,0.298920,0.000000,...,0.000000,0.000000,0.522512,0.095343,0.242455,0.112508,0.815429,0.370586,8,H50%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3848,0.284930,0.730358,0.496297,0.000000,0.000000,0.418072,0.223871,0.110502,0.000000,0.545941,...,0.468734,0.000000,0.572904,0.000000,0.221457,0.347038,0.202071,0.592896,12,K_Control
3849,0.257077,0.000000,0.000000,0.385557,0.231760,0.276198,0.296320,0.355897,0.000000,0.000000,...,0.257204,0.000000,0.646378,0.271761,0.163825,0.252310,0.610014,0.497658,12,Control
3850,0.244874,0.557300,0.632031,0.000000,0.000000,0.442473,0.000000,0.000000,0.000000,1.093996,...,0.690550,0.000000,0.191527,0.000000,0.196538,0.611789,0.000000,1.074702,12,H50%
3851,0.410709,0.060702,0.085851,0.432054,0.190134,0.301681,0.228655,0.293132,0.000000,0.000000,...,0.074983,0.000000,0.798612,0.717082,0.003022,0.049513,0.452348,0.603918,12,H75%


In [30]:
df.Class.unique

<bound method Series.unique of 0       N_Control
1       P_Control
2       K_Control
3         Control
4            H50%
          ...    
3848    K_Control
3849      Control
3850         H50%
3851         H75%
3852      Control
Name: Class, Length: 3853, dtype: object>

In [20]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

##SELECCION DE N Y CLASES
N_data = df[(df['Class']=='N_Control') | (df['Class']=='N_Exceso') | (df['Class']=='N_Deficiencia')]

In [ ]:
a = ['29_marzo',
 '14_abril',
 '28_abril',
 '7_mayo',
 '12_mayo',
 '19_mayo',
 '26_mayo',
 '2_junio',
 '11_junio',
 '16_junio',
 '23_junio',
 '2_julio',
 '9_julio',
 '14_julio',
 '23_julio',
 '5_agosto',
 '13_agosto',
 '19_agosto',
 '15_setiembre',
 '24_setiembre',
 '15_octubre',
 '29_octubre',
 '12_noviembre',
 '26_noviembre']


In [21]:
N_data = N_data.iloc[:,:-1]
n2clas={'N_Deficiencia':0, 'N_Control':1, 'N_Exceso':2}
N_data.Class.replace(n2clas, inplace=True)
N_data

,meta_u_0,meta_u_1,meta_u_2,meta_u_3,meta_u_4,meta_u_5,meta_u_6,meta_u_7,meta_u_8,meta_u_9,...,meta_sig_11,meta_sig_12,meta_sig_13,meta_sig_14,meta_sig_15,meta_sig_16,meta_sig_17,meta_sig_18,meta_sig_19,Class
0,0.078699,0.000000,0.142968,0.291437,0.194801,0.252386,0.280699,0.194773,0.000000,0.000000,...,0.174845,0.003038,0.000000,0.586983,0.000000,0.354402,0.142688,0.515514,0.648625,1
6,0.649001,0.000000,0.000000,0.615803,0.421148,0.275630,0.265670,0.545408,0.000000,0.000000,...,0.111564,0.183654,0.000000,0.701441,0.546494,0.261433,0.090762,0.666555,0.673366,1
19,0.422982,0.000000,0.000000,0.588879,0.278483,0.111502,0.279465,0.415425,0.000000,0.000000,...,0.030619,0.000000,0.000000,0.761748,0.787005,0.147187,0.000000,0.730097,0.563534,1
26,0.514778,0.000000,0.000000,0.598868,0.368656,0.345593,0.298195,0.460295,0.000000,0.000000,...,0.146196,0.070085,0.000000,0.712485,0.481750,0.348905,0.146946,0.722736,0.691882,1
33,0.567048,0.000000,0.000000,1.410952,0.837366,0.176433,0.386615,0.941868,0.509762,0.000000,...,0.182912,0.000000,0.128513,0.954945,1.322860,0.141742,0.000000,0.985269,0.390527,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3807,0.018741,0.217064,0.467396,0.000000,0.062320,0.687517,0.432002,0.095036,0.161232,0.367051,...,0.293208,0.507756,0.398323,0.000000,0.000000,0.240750,0.578494,0.000000,0.047932,2
3813,0.288991,0.499323,0.505346,0.000000,0.293389,0.284323,0.442379,0.035534,0.297361,0.222238,...,0.346535,0.578142,0.808017,0.000000,0.433672,0.255369,0.644090,0.000000,0.000000,2
3820,0.000000,0.721509,0.912714,0.000000,0.000000,0.597034,0.380042,0.000000,0.000000,0.845107,...,0.000000,0.657654,0.037090,0.000000,0.000000,0.110582,0.641064,0.000000,0.139485,2
3827,0.000000,0.292338,0.383014,0.105401,0.108881,0.272453,0.499986,0.109610,0.211929,0.000000,...,0.187697,0.052298,0.203316,0.359443,0.216545,0.046332,0.000000,0.242809,0.116265,2


In [23]:
def train(idx, ep = 100):
    idx_result = {'idx_used':copy(idx), 'f1': 0, 'acc':0}
    idx_copy = copy(idx)
    aux_f1_idx = []
    aux_ac_idx = []
    n_f = len(idx_copy)
    idx_copy.extend([-1])
    a = datapaltas(df = N_data.iloc[:, idx_copy], y_idx=-1)
    EPOCHS = ep
    
    skf = StratifiedKFold()
    ## y_idx -2
    skf.get_n_splits(N_data, N_data.iloc[:,-1])
    for fold, (train_ids, test_ids) in enumerate(skf.split(N_data, N_data.iloc[:,-1])):
        l_v = []
        a_v = []
        f_v = []
        #print(f'Fold {fold+1}')
        #print('------------------------')
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_loader = torch.utils.data.DataLoader(
                          a, 
                          batch_size=256, sampler=train_subsampler)
        test_loader = torch.utils.data.DataLoader(
                          a,
                          batch_size=1, sampler=test_subsampler)
        model = SimpleClassification(n_features=n_f)
        model.to(device)
        #print(model)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 0.001)
        itt = tqdm(range(EPOCHS))
        for i in itt:
            loss_epoch = 0
            acc_train, acc_test = 0.0, 0.0
            f1_train, f1_test = 0.0, 0.0
            model.train()
            for X, y in train_loader:
                X, y = X.to(device), y.to(device).flatten()
                optimizer.zero_grad()
                y_pred = model(X)
                loss = criterion(y_pred, y)
                acc = multi_acc(y_pred, y)
                f1 = F1_score(y_pred, y)
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
                acc_train += acc.item()
                f1_train += f1.item()

            with torch.no_grad():

                for X, y in test_loader:
                    X, y = X.to(device), y.to(device).flatten()
                    model.eval()
                    output = model(X)
                    acc = multi_acc(output, y)
                    f1 = F1_score(output, y)
                    acc_test += acc.item()
                    f1_test += f1.item()

            itt.set_description("Acc train: %.2f Acc test: %.2f F1 train: %.2f F1 test: %.2f" % (acc_train/len(train_loader), acc_test/len(test_loader), f1_train/len(train_loader), f1_test/len(test_loader)))

            l_v.append(loss_epoch)
            a_v.append(acc_test/len(test_loader))
            f_v.append(f1_test/len(test_loader))
            
            #print(f'Loss {i+1}: {loss_epoch}')
            #print(f'Acc {i+1}: {acc_epoch/len(train_loader)}')
            #print(f'F1 {i+1}: {f1_epoch/len(train_loader)}')
        #print(a_v, f_v)
        aux_f1_idx.append(f_v)
        aux_ac_idx.append(a_v)
        
    aux_ac_idx = np.array(aux_ac_idx)
    aux_f1_idx = np.array(aux_f1_idx)
    #print(aux_ac_idx, aux_f1_idx)
    idx_result['f1'] = max(np.mean(aux_f1_idx, axis = 0))
    idx_result['acc'] = max(np.mean(aux_ac_idx, axis = 0))
    return idx_result
        #plt.figure(), plt.plot(l_v), plt.title('Loss by epoch')
        #plt.figure(), plt.plot(a_v), plt.title('Accuracy test by epoch')
        #plt.figure(), plt.plot(f_v), plt.title('F1 test by epoch')
        #plt.show()
    
N_FEATURES = 40



###SELECTOR
def select_k_best(k = 10, ev = 'f1'):
    """
    ev = 'acc' or 'f1'
    """
    total_log = []
    list_features = list(range(N_FEATURES))
    initial_feat = []
    while(len(initial_feat)<k):
        selector = {}
        for i in list_features:
            initial_feat.append(i)
            print(f"=======================================Using {initial_feat} ===============")
            res = train(initial_feat, ep = 100)
            initial_feat.pop()
            selector.update({i:res[ev]})
            total_log.append(res)
        best_feature = max(selector, key=selector.get)
        initial_feat.append(best_feature)
        list_features.remove(best_feature)
        print("================================================")
        print(f"=================ADEDD {max(selector, key=selector.get)} TO BEST FEATURES ==========")
        print(f"================={ev} = {selector[best_feature]} ================================")
        
    print(f"FINAL FEATURES")
    print(initial_feat)
    return total_log

log = select_k_best(k = 20)

with open('log.txt', mode='w') as f:
    f.write(str(log))

=======================================Using [0] ===============


Acc train: 43.00 Acc test: 36.08 F1 train: 0.42 F1 test: 0.36: 100%|█████████████████| 100/100 [00:19<00:00,  5.16it/s]
Acc train: 41.50 Acc test: 40.62 F1 train: 0.39 F1 test: 0.41: 100%|█████████████████| 100/100 [00:19<00:00,  5.15it/s]
Acc train: 41.50 Acc test: 31.25 F1 train: 0.40 F1 test: 0.31: 100%|█████████████████| 100/100 [00:19<00:00,  5.19it/s]
Acc train: 42.00 Acc test: 33.33 F1 train: 0.42 F1 test: 0.33: 100%|█████████████████| 100/100 [00:19<00:00,  5.11it/s]
Acc train: 43.50 Acc test: 30.21 F1 train: 0.41 F1 test: 0.30: 100%|█████████████████| 100/100 [00:19<00:00,  5.20it/s]


=======================================Using [1] ===============


Acc train: 41.50 Acc test: 37.11 F1 train: 0.32 F1 test: 0.37: 100%|█████████████████| 100/100 [00:19<00:00,  5.17it/s]
Acc train: 42.00 Acc test: 37.50 F1 train: 0.41 F1 test: 0.38: 100%|█████████████████| 100/100 [00:19<00:00,  5.13it/s]
Acc train: 43.00 Acc test: 38.54 F1 train: 0.39 F1 test: 0.39: 100%|█████████████████| 100/100 [00:19<00:00,  5.23it/s]
Acc train: 47.00 Acc test: 30.21 F1 train: 0.38 F1 test: 0.30: 100%|█████████████████| 100/100 [00:19<00:00,  5.14it/s]
Acc train: 42.50 Acc test: 43.75 F1 train: 0.36 F1 test: 0.44: 100%|█████████████████| 100/100 [00:19<00:00,  5.17it/s]


=======================================Using [2] ===============


...

=======================================Using [23, 38] ===============


Acc train: 57.00 Acc test: 45.36 F1 train: 0.57 F1 test: 0.45: 100%|█████████████████| 100/100 [00:19<00:00,  5.18it/s]
Acc train: 50.50 Acc test: 42.71 F1 train: 0.50 F1 test: 0.43: 100%|█████████████████| 100/100 [00:19<00:00,  5.14it/s]
Acc train: 59.50 Acc test: 41.67 F1 train: 0.59 F1 test: 0.42: 100%|█████████████████| 100/100 [00:19<00:00,  5.18it/s]
Acc train: 53.00 Acc test: 44.79 F1 train: 0.53 F1 test: 0.45: 100%|█████████████████| 100/100 [00:19<00:00,  5.16it/s]
Acc train: 53.50 Acc test: 35.42 F1 train: 0.54 F1 test: 0.35: 100%|█████████████████| 100/100 [00:19<00:00,  5.09it/s]


=======================================Using [23, 39] ===============


Acc train: 59.50 Acc test: 37.11 F1 train: 0.59 F1 test: 0.37: 100%|█████████████████| 100/100 [00:20<00:00,  4.94it/s]
Acc train: 53.00 Acc test: 53.12 F1 train: 0.53 F1 test: 0.53: 100%|█████████████████| 100/100 [00:19<00:00,  5.13it/s]
Acc train: 54.00 Acc test: 42.71 F1 train: 0.53 F1 test: 0.43: 100%|█████████████████| 100/100 [00:20<00:00,  4.85it/s]
Acc train: 53.00 Acc test: 41.67 F1 train: 0.53 F1 test: 0.42:  55%|█████████▉        | 55/100 [00:12<00:10,  4.38it/s]


KeyboardInterrupt: 